# PLSA

In [1]:
import numpy as np

# Define the number of topics
num_topics = 5

# Define the number of iterations
num_iterations = 100

c:\anaconda\envs\machine_learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Load the document-term matrix
doc_term_matrix = np.array([[1, 0, 2, 3, 0],
                            [0, 1, 1, 0, 1],
                            [2, 0, 3, 1, 0],
                            [0, 1, 2, 0, 1],
                            [1, 1, 0, 1, 1]])

# Normalize the document-term matrix
doc_term_matrix_norm = doc_term_matrix / np.sum(doc_term_matrix, axis=1, keepdims=True)



In [3]:
# Initialize the topic-word probability matrix
topic_word_prob = np.random.rand(num_topics, doc_term_matrix.shape[1])
topic_word_prob /= np.sum(topic_word_prob, axis=1, keepdims=True)



In [4]:
# Initialize the document-topic probability matrix
doc_topic_prob = np.random.rand(doc_term_matrix.shape[0], num_topics)
doc_topic_prob /= np.sum(doc_topic_prob, axis=1, keepdims=True)




In [5]:
# Define the number of iterations
num_iterations = 100
# Iterate between the E-step and M-step
for i in range(num_iterations):
    # E-step: compute the expected counts
    expected_counts = np.zeros((doc_term_matrix.shape[0], num_topics, doc_term_matrix.shape[1]))
    for d in range(doc_term_matrix.shape[0]):
        for w in range(doc_term_matrix.shape[1]):
            for z in range(num_topics):
                expected_counts[d, z, w] = doc_term_matrix_norm[d, w] * topic_word_prob[z, w] * doc_topic_prob[d, z]
        expected_counts[d] /= np.sum(expected_counts[d])
    
    # M-step: update the topic-word and document-topic probabilities
    for z in range(num_topics):
        for w in range(doc_term_matrix.shape[1]):
            topic_word_prob[z, w] = np.sum(expected_counts[:, z, w])
        topic_word_prob[z] /= np.sum(topic_word_prob[z])
    for d in range(doc_term_matrix.shape[0]):
        for z in range(num_topics):
            doc_topic_prob[d, z] = np.sum(expected_counts[d, z])
        doc_topic_prob[d] /= np.sum(doc_topic_prob[d])

# Print the topic-word probability matrix
print("Topic-Word Probability Matrix:\n", topic_word_prob)

# Print the document-topic probability matrix
print("\nDocument-Topic Probability Matrix:\n", doc_topic_prob)


Topic-Word Probability Matrix:
 [[4.31198384e-046 4.30226891e-219 1.68669983e-017 1.00000000e+000
  2.08213512e-219]
 [9.73230874e-003 3.66336819e-001 0.00000000e+000 2.31110680e-001
  3.92820192e-001]
 [0.00000000e+000 1.67056688e-001 7.29502906e-001 0.00000000e+000
  1.03440405e-001]
 [3.55457100e-002 3.72169264e-003 0.00000000e+000 7.42347021e-001
  2.18385577e-001]
 [3.31904248e-052 2.35866534e-051 1.00000000e+000 5.98228953e-079
  5.07919422e-053]]

Document-Topic Probability Matrix:
 [[1.00000000e+000 2.24810345e-061 5.24024810e-033 1.95096121e-013
  2.12614935e-017]
 [0.00000000e+000 2.12044010e-013 8.09904250e-001 2.37231219e-063
  1.90095750e-001]
 [3.43837754e-045 5.47547610e-107 3.94922776e-016 1.14807150e-057
  1.00000000e+000]
 [0.00000000e+000 4.87292719e-043 1.21289738e-007 1.50011449e-092
  9.99999879e-001]
 [4.87889565e-003 6.42898779e-001 5.84641050e-056 3.52222326e-001
  0.00000000e+000]]
